In [3]:
%load_ext autoreload
%autoreload 2

from itertools import product
import sys
sys.path.append('/home/projects/nyosef/zvise/PixelGen/')
from PixelGen.multimodalvi import MultiModalSCVI
from PixelGen.multimodalvae import MultiModalVAE, AggMethod, D
from PixelGen.enums import AggMethod, D
from PixelGen.metrics import MultiModalVIMetrics
from sklearn.preprocessing import PowerTransformer
from pathlib import Path


import anndata as ad
import pixelator
import torch
import scvi
import scipy
# from scvi import autotune

import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

# import ray
# from ray import tune


from PixelGen.pxl_utils import train_model, get_model_latents
from PixelGen.scvi_utils import plot_losses, pca_neighbors_umap, calc_PCA
from pixelator.common.statistics import clr_transformation, dsb_normalize


from pixelator.pna.plot import molecule_rank_plot
# from pixelator.plot import molecule_rank_plot, cell_count_plot, scatter_umi_per_upia_vs_tau
# from pixelator.statistics import c
# lr_transformation
# from pixelator.analysis.normalization import dsb_normalize


from sklearn.preprocessing import StandardScaler, MinMaxScaler 

from PixelGen.pxl_utils import train_model, get_model_latents, convert_polarization_to_feature_matrix, \
     convert_colocalization_to_feature_matrix, download_pxl
from PixelGen.scvi_utils import plot_losses, pca_neighbors_umap, calc_PCA, add_one_hot_encoding_obsm, plot_cumulative_variance
from PixelGen.common_utils import standardize, std_clip, filter_hv, split_pair_column, filter_df_by_two_columns, rank_plot
from PixelGen.metrics import MultiModalVIMetrics, distr_autocorrelation_in_latent
from PixelGen.multimodalvi import MultiModalSCVI
from PixelGen.multimodalvae import MultiModalVAE, AggMethod, D
from PixelGen.enums import AggMethod, D

import tempfile

from scvi import REGISTRY_KEYS
from scvi.module.base import (
    BaseModuleClass,
    LossOutput,
    PyroBaseModuleClass,
    auto_move_data,
)
from torch.distributions import NegativeBinomial, Normal, Poisson, MixtureSameFamily, Beta
from torch.distributions import kl_divergence as kl



# from cytovi import CytoVI

print(torch.cuda.is_available())
from sklearn.decomposition import PCA

from anndata import AnnData
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)
sc.set_figure_params(figsize=(6, 6), frameon=False)

sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

from utils import plot_latent, plot_gene_heatmap, plot_model_latents
from doublet_seperation import B_CD4_logfc_dict, B_CD8_logfc_dict, run_cellwise_coloc_analysis_to_disk, concat_abundance_to_adata, add_doublets_metadata
%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"
from pixelator import read_pna as read
import pickle
import scipy.sparse as sp
from scipy.sparse.csgraph import dijkstra
import anndata
import hotspot

import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import igraph as ig
import leidenalg
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm import tqdm

Seed set to 0


True
Last run with scvi-tools version: 1.3.3


In [4]:
ANNOTATED_ADATA_PATH='/home/projects/nyosef/zvise/PixelGen/PixelGen/Data/adatas/final_adatas/adata_annotated.h5ad'
DATA_DIR = Path("/home/projects/nyosef/zvise/PxlgnProject/Data")
DOUBLETS_SEP_CACHE='/home/projects/nyosef/zvise/PixelGen/PixelGen/cache/doublet_separation_run'
SEPARATION_PKL_PATH= '/home/projects/nyosef/zvise/PixelGen/PixelGen/cache/results.pkl'
T_CELLS_ADATA_PATH='/home/projects/nyosef/zvise/PixelGen/PixelGen/Data/adatas/final_adatas/t_cell_adata.h5ad'


files = [f for f in DATA_DIR.rglob('*.pxl') if f.is_file()]
pg_data = read(files)
adata=sc.read_h5ad(ANNOTATED_ADATA_PATH)

In [5]:
cell=adata.obs_names[0]
# edge_list=pg_data.filter(components=cell).edgelist().to_df()
df_pixels=pg_data.filter(components=cell).precomputed_layouts().to_df()
df_pixels.head()

,index,pixel_type,x,y,z,component,graph_projection,layout,sample,B2M,...,CD328,IgD,CD159a,FMC63,CD64,CD20,CD163,CD159c,NKp80,CD16
0,8238457035169063,A,5.248064e+07,-6.485522e+07,-4.213469e+07,d2d7a08c8b4a02a1,full,wpmds_3d,PNA066_CARTcells_S01_S1,1,...,0,0,0,0,0,0,0,0,0,0
1,30914261206784358,A,-3.424900e+06,-4.029121e+07,-4.825244e+07,d2d7a08c8b4a02a1,full,wpmds_3d,PNA066_CARTcells_S01_S1,0,...,0,0,0,0,0,0,0,0,0,0
2,61797079594469301,A,-5.872723e+07,2.344133e+07,-5.205771e+07,d2d7a08c8b4a02a1,full,wpmds_3d,PNA066_CARTcells_S01_S1,0,...,0,0,0,0,0,0,0,0,0,0
3,42425746382690005,A,-8.132270e+07,5.632060e+06,-4.021235e+07,d2d7a08c8b4a02a1,full,wpmds_3d,PNA066_CARTcells_S01_S1,0,...,0,0,0,0,0,0,0,0,0,0
4,46349312809430673,A,-8.131929e+07,-4.111317e+07,-1.027522e+07,d2d7a08c8b4a02a1,full,wpmds_3d,PNA066_CARTcells_S01_S1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def run_hotspot(layout, top_n=70):
    # 1. Identify marker columns
    marker_cols = [c for c in layout.columns if c not in 
               ['index',"pixel_type","x","y","z","component","graph_projection","layout","sample"]]
    
    # 2. Extract and Standardize Data (Z-Score)
    raw_data = layout[marker_cols].values.astype(float)
    mean = np.mean(raw_data, axis=0)
    std = np.std(raw_data, axis=0)
    
    # Filter valid markers (std > 0)
    valid_mask = std > 0
    raw_data = raw_data[:, valid_mask]
    mean = mean[valid_mask]
    std = std[valid_mask]
    # Keep track of which markers are valid
    valid_marker_names = np.array(marker_cols)[valid_mask]
    
    # Z-score and shift by epsilon to avoid "all zero" error
    standardized_data = ((raw_data - mean) / std) + 1e-6
    
    # 3. Create AnnData
    pixel_adata = AnnData(standardized_data)
    pixel_adata.var_names = valid_marker_names
    pixel_adata.obs = layout[["component","sample"]].copy()
    pixel_adata.obsm["spatial"] = layout[["x","y","z"]].values
    
    # 4. Initialize Hotspot (model='none' expects standardized data)
    hs = hotspot.Hotspot(pixel_adata, model='none', latent_obsm_key="spatial")
    hs.create_knn_graph(n_neighbors=30)
    
    # 5. Compute Autocorrelations (Fast O(N))
    # jobs=-1 uses all CPU cores
    pol = hs.compute_autocorrelations(jobs=-1).reset_index().rename(columns={"Gene":"marker","C":"pol_c","Z":"pol_z"})
    
    # --- FEATURE SELECTION STEP ---
    # Select only the top N markers with the strongest spatial pattern (highest Z-score)
    # We maintain the list so we know what to compute next
    significant_markers = pol.sort_values('pol_z', ascending=False).head(top_n)['marker'].tolist()
    
    # 6. Compute Pairwise Correlations (Expensive Step)
    # ONLY run on the significant list. This is what makes it fast.
    _ = hs.compute_local_correlations(genes=significant_markers, jobs=-1)

    # 7. Format Output
    coloc_c = (
        hs.local_correlation_c
        .reset_index()
        .rename(columns={"index": "marker"})
        .melt(id_vars="marker", var_name="marker2", value_name="coloc_c")
    )
    
    coloc_z = (
        hs.local_correlation_z
        .reset_index()
        .rename(columns={"index": "marker"})
        .melt(id_vars="marker", var_name="marker2", value_name="coloc_z")
    )

    coloc = coloc_c.merge(coloc_z, on=["marker","marker2"])
    
    # Create the matrix only for the selected markers
    coloc_matrix = coloc.pivot(index="marker", columns="marker2", values="coloc_c")
    
    # Fill diagonal with autocorrelation values
    # Reindex pol to match the current selection
    diag_vals = pol.set_index('marker')['pol_c'].reindex(coloc_matrix.index)
    np.fill_diagonal(coloc_matrix.values, diag_vals)
    
    # Flatten to "Marker1/Marker2" format
    rows, cols = np.triu_indices(len(coloc_matrix), k=0)
    
    flat_df = pd.DataFrame(
        [coloc_matrix.values[rows, cols]], 
        columns=coloc_matrix.index[rows] + '/' + coloc_matrix.columns[cols]
    )
    
    # Add component ID so we can merge later
    flat_df.index = [layout['component'].iloc[0]]
    
    return flat_df

In [7]:
ש

100%|██████████| 149/149 [00:01<00:00, 117.76it/s]


Computing pair-wise local correlation on 70 features...


100%|██████████| 2415/2415 [00:15<00:00, 159.28it/s]


,B2M/B2M,B2M/CD102,B2M/CD117,B2M/CD11b,B2M/CD11c,B2M/CD127,B2M/CD13,B2M/CD134,B2M/CD150,B2M/CD152,...,TCRva7.2/TCRva7.2,TCRva7.2/TIGIT,TCRva7.2/mIgG2a,TCRva7.2/mIgG2b,TIGIT/TIGIT,TIGIT/mIgG2a,TIGIT/mIgG2b,mIgG2a/mIgG2a,mIgG2a/mIgG2b,mIgG2b/mIgG2b
d2d7a08c8b4a02a1,0.006628,-0.004174,0.000388,0.000673,0.00024,-0.000644,0.001025,-0.000389,0.001481,-0.001962,...,0.010102,-0.000378,-0.000448,-0.000404,0.004902,-0.000206,-0.000186,0.003425,-0.00022,0.004259


In [11]:
import os
CACHE_PATH = "/home/projects/nyosef/zvise/PixelGen/PixelGen/cache/HOTSPOT/hotspot_cache.pkl"


def load_cache(path=CACHE_PATH):
    return pickle.load(open(path, "rb")) if os.path.exists(path) else {}


def save_cache(cache, path=CACHE_PATH):
    pickle.dump(cache, open(path, "wb"))


def run_hotspot_cached(cell_ids, pg_data, batch=50):
    """
    Wrapper for your run_hotspot(layout) function.
    - Loads previous cache
    - Skips completed cells
    - Saves every `batch` cells
    - Returns combined DataFrame
    """
    cache = load_cache()
    todo = [cid for cid in cell_ids if cid not in cache]

    print(f"Total: {len(cell_ids)} | Cached: {len(cache)} | Remaining: {len(todo)}")

    for i, cid in enumerate(tqdm(todo)):
        try:
            df = pg_data.filter(components=cid).precomputed_layouts().to_df()
            if df.empty:
                continue

            flat_row = run_hotspot(df)   # <--- **your function**
            cache[cid] = flat_row.iloc[0]  # store as Series

        except Exception as e:
            print(f"⚠️ Error on {cid}: {e}")
            continue

        # Save periodically
        if (i + 1) % batch == 0:
            save_cache(cache)

    save_cache(cache)  # final save

    return pd.DataFrame.from_dict(cache, orient="index").fillna(0)


In [ ]:
cell_ids = list(adata.obs_names)

results = run_hotspot_cached(cell_ids, pg_data, batch=50)

results.to_pickle("/home/projects/nyosef/zvise/PixelGen/PixelGen/cache/HOTSPOT/hotspot_final.pkl")

Total: 2338 | Cached: 2253 | Remaining: 85


100%|██████████| 147/147 [00:00<00:00, 273.12it/s]


Computing pair-wise local correlation on 70 features...


  0%|          | 0/85 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [13]:
results

,B2M/B2M,B2M/CD102,B2M/CD117,B2M/CD11b,B2M/CD13,B2M/CD134,B2M/CD150,B2M/CD152,B2M/CD156c,B2M/CD159c,...,HLA-DR/mIgG2a,HLA-DR-DP-DQ/mIgG2a,KLRG1/TCRgd,KLRG1/mIgG2a,TCRVB5/TCRgd,TCRVB5/mIgG2a,TCRgd/TCRva7.2,TCRgd/mIgG2a,TCRva7.2/mIgG2a,mIgG2a/mIgG2a
d2d7a08c8b4a02a1,0.006700,-0.004149,0.000356,0.000657,0.001039,-0.000382,0.001391,-0.001984,-0.000989,-0.001839,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ab17222ee789b183,0.011672,-0.006085,0.000000,0.000000,0.000000,0.000000,0.000381,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
009a143087090be1,0.010722,-0.001407,0.000000,-0.001696,0.000000,0.000000,-0.002000,-0.002850,0.000000,-0.002469,...,-0.001503,-0.000734,-0.000093,-0.000196,-0.000058,-0.000123,-0.000095,-0.000115,-0.000199,0.022187


OLD VERSION

In [160]:
# def run_hotspot(layout):
#     marker_cols = [c for c in layout.columns if c not in 
#                ['index',"pixel_type","x","y","z","component","graph_projection","layout","sample"]]
    
#     pixel_adata = AnnData(layout[marker_cols].values)
#     pixel_adata.var_names = marker_cols
#     pixel_adata.obs = layout[["component","sample"]].copy()
#     pixel_adata.obsm["spatial"] = layout[["x","y","z"]].values
    
#     umi_counts = np.ones(pixel_adata.n_obs, dtype=int)  
#     pixel_adata.obs["umi_counts"] = umi_counts
    
#     hs = hotspot.Hotspot(
#     pixel_adata,
#     model='none',
#     latent_obsm_key="spatial",     # use x,y,z as latent embedding
#     umi_counts_obs_key='umi_counts'        # we do not use UMI model
#     )

#     # Build KNN graph on XYZ
#     hs.create_knn_graph(n_neighbors=30)
    
#     pol = hs.compute_autocorrelations().reset_index().rename(columns={"Gene":"marker","C":"pol_c","Z":"pol_z"})
    
#     _ = hs.compute_local_correlations(genes=pixel_adata.var_names)


#     coloc_c = (
#         hs.local_correlation_c
#         .reset_index()
#         .rename(columns={"index": "marker"})
#         .melt(id_vars="marker", var_name="marker2", value_name="coloc_c")
#     )

#     # Z version
#     coloc_z = (
#         hs.local_correlation_z
#         .reset_index()
#         .rename(columns={"index": "marker"})
#         .melt(id_vars="marker", var_name="marker2", value_name="coloc_z")
#     )

#     coloc = coloc_c.merge(coloc_z, on=["marker","marker2"])

#     coloc_matrix = coloc.pivot(index="marker", columns="marker2", values="coloc_c")
    
#     diag_vals = pol.set_index('marker')['pol_c'].reindex(coloc_matrix.index)
#     np.fill_diagonal(coloc_matrix.values, diag_vals)
    
#     rows, cols = np.triu_indices(len(coloc_matrix), k=0)

#     flat_df = pd.DataFrame(
#         [coloc_matrix.values[rows, cols]], 
#         columns=coloc_matrix.index[rows] + '/' + coloc_matrix.columns[cols]
#     )
    
#     return flat_df